#### 사용자 정의 모델 클래스
- 부모클래스 : nn.module
- 필수오버라이딩
    * __ init __(): 모델 층 구성 즉, 설계
    * forward(): 순방향 학습 진행 코드 구현

In [1]:
## 모듈 로딩
import torch                                # 텐서 관련 모듈
import torch.nn as nn                       # 인공신경망 관련 모듈
import torch.nn.functional as F             # 인공신경망 관련 함수들 모듈 ( 손실함수, 활성화 함수 등등 )
import torch.optim as optim                 # 최적화 관련 모듈 ( 가중치, 절편 빠르게 찾아주는 알고리즘 )
from torchinfo import summary               # 모델 구조 및 정보 관련 모듈
from torchmetrics.regression import *       # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *   # 분류 성능 지표 관련 모듈

In [2]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

- [기본] 신경망클래스 <hr>
    * 입력층 - 입력 피쳐 고정
    * 출력층 - 출력 타겟 고정
    * 은닉층 - 고정

In [3]:
# 모델설계
# 데이터셋 : 피쳐 4개,   타겟 1개,   회귀
# 입 력 층 : 입력 4개    출력 20개   AF ReLU
# 은 닉 층 : 입력 20개   출력 100개  AF ReLU
# 출 력 층 : 입력 100개  출력 1개    AF X, (분류 -> Sigmoid(2진) & Softmax(다중))

In [4]:
class MyModel(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 callback func)
    def __init__(self):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer=nn.Linear(4, 20)       # w: 4 + b: 1 => 1P(Perceptron), 5 * 20 = 100개 변수
        self.hidden_layer=nn.Linear(20, 100)    # w: 20 + b: 1 => 1P, 21 * 100 = 2100개 변수
        self.output_layer=nn.Linear(100, 1)     # w: 100 + b: 1 => 1P, 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, dataset):
        print('calling forward()')
        y=self.input_layer(dataset) # 1개 퍼셉트론 : y=x1w1+x2w2+x3w3+x4w4+b
        y=F.relu(y)                   # 0<=y ----> 죽은 ReLU ==> LeakyReLU

        y=self.hidden_layer(y) # y=x1w1+y2w2+~~~+x20w20+b
        y=F.relu(y)

        return self.output_layer(y) # y=x1w1+y2w2+~~~+x100w100+b

In [5]:
# 입력 피쳐 수가 동적인 모델
class MyModel2(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 callback func)
    def __init__(self, in_feature):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer=nn.Linear(in_feature, 20)       # w: 4 + b: 1 => 1P(Perceptron), 5 * 20 = 100개 변수
        self.hidden_layer=nn.Linear(20, 100)    # w: 20 + b: 1 => 1P, 21 * 100 = 2100개 변수
        self.output_layer=nn.Linear(100, 1)     # w: 100 + b: 1 => 1P, 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, dataset):
        print('calling forward()')
        y=self.input_layer(dataset) # 1개 퍼셉트론 : y=x1w1+x2w2+x3w3+x4w4+b
        y=F.relu(y)                 # 0<=y ----> 죽은 ReLU ==> LeakyReLU

        y=self.hidden_layer(y) # y=x1w1+y2w2+~~~+x20w20+b
        y=F.relu(y)

        return self.output_layer(y) # y=x1w1+y2w2+~~~+x100w100+b

In [6]:
# 입력 피쳐 수, 은닉층 퍼셉트론 수가 동적인 모델
class MyModel3(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 callback func)
    def __init__(self, in_feature, in_out, h_out):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer=nn.Linear(in_feature, in_out)  # w: in_feature 수 + b: 1 => 1P(Perceptron), 5 * 20 = 100개 변수
        self.hidden_layer=nn.Linear(in_out, h_out)      # w: in_out 수 + b: 1 => 1P, 21 * 100 = 2100개 변수
        self.output_layer=nn.Linear(h_out, 1)           # w: h_out 수 + b: 1 => 1P, 101 * 1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, dataset):
        print('calling forward()')
        y=self.input_layer(dataset) # 1개 퍼셉트론 : y=x1w1+x2w2+x3w3+x4w4+b
        y=F.relu(y)                   # 0<=y ----> 죽은 ReLU ==> LeakyReLU
        
        y=self.hidden_layer(y) # y=x1w1+y2w2+~~~+x20w20+b
        y=F.relu(y)

        return self.output_layer(y) # y=x1w1+y2w2+~~~+x100w100+b

In [7]:
# 은닉층의 개수가 동적인 모델
class MyModel4(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 callback func)
    def __init__(self, in_feature, in_out, h_out, hidden_layers_num=1):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer=nn.Linear(in_feature, in_out)
        self.hidden_layers=nn.ModuleDict([nn.Linear(in_out, h_out) for _ in range(hidden_layers_num)])
        # self.hidden_layer=nn.Linear(in_out, h_out)
        self.output_layer=nn.Linear(h_out, 1)
        
    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, dataset):
        print('calling forward()')
        y=self.input_layer(dataset) # 1개 퍼셉트론 : y=x1w1+x2w2+x3w3+x4w4+b
        y=F.relu(y)                   # 0<=y ----> 죽은 ReLU ==> LeakyReLU
        
        y=self.hidden_layer(y) # y=x1w1+y2w2+~~~+x20w20+b
        y=F.relu(y)

        return self.output_layer(y) # y=x1w1+y2w2+~~~+x100w100+b

In [8]:
## 모델 인스턴스 생성
# ml=MyModel()
# ml=MyModel2(4)
ml=MyModel3(4, 50, 30)

# ml=MyModel4(4, 50, 30)
# ml=MyModel4(4, 50, 30, hidden_layers_num=3)

In [9]:
# 모델 파라미터 즉, W와 b확인
for n in ml.named_parameters(): print(n)

('input_layer.weight', Parameter containing:
tensor([[ 0.2576, -0.2207, -0.0969,  0.2347],
        [-0.4707,  0.2999, -0.1029,  0.2544],
        [ 0.0695, -0.0612,  0.1387,  0.0247],
        [ 0.1826, -0.1949, -0.0365, -0.0450],
        [ 0.0725, -0.0020,  0.4371,  0.1556],
        [-0.1862, -0.3020, -0.0838, -0.2157],
        [-0.1602,  0.0239,  0.2981,  0.2718],
        [-0.4888,  0.3100,  0.1397,  0.4743],
        [ 0.3300, -0.4556, -0.4754, -0.2412],
        [ 0.4391, -0.0833,  0.2140, -0.2324],
        [ 0.4906, -0.2115,  0.3750,  0.0059],
        [-0.2634,  0.2570, -0.2654,  0.1471],
        [-0.1444, -0.0548, -0.4807, -0.2384],
        [ 0.2713, -0.1215,  0.4980,  0.4008],
        [-0.0234, -0.3337,  0.3045,  0.1552],
        [-0.3232,  0.3248,  0.3036,  0.4434],
        [-0.2803, -0.0823, -0.0097,  0.0730],
        [-0.3795, -0.3548,  0.2720, -0.1172],
        [ 0.2442,  0.0285,  0.1642,  0.1099],
        [ 0.1818,  0.2479, -0.4631,  0.2517],
        [-0.3516, -0.3773,  0.0304,

In [10]:
# 학습 진행 ==> 모델인스턴스명 (데이터)
# 임의의 데이터
dataTS=torch.FloatTensor([[1,3,5,7], [2,4,6,8]])
targetTS=torch.FloatTensor([[4],[5]])

# 학습
pre_y = ml(dataTS)
print(pre_y)

calling forward()
tensor([[0.3364],
        [0.3836]], grad_fn=<AddmmBackward0>)
